# Module 4.2: Runtime Basics in LangGraph

**Prerequisites:**
- Module 4.1: Short-term memory (conversation context)

**What you'll learn:**
- 🎯 What is Runtime?
- 📝 Context: Passing information to agents
- 🔧 ToolRuntime: Accessing context in tools
- 🎨 Dynamic prompts based on context
- 🔌 Middleware: Pre/post model hooks
- 💼 Production patterns

**Real HR Use Case:**
Build an HR assistant that:
- Knows which user is asking (user context)
- Has access to company info (org context)
- Adapts responses based on user role
- Logs requests for audit trails

**Why Runtime Matters:**
Runtime provides the **environment** your agent runs in - like knowing:
- Who is asking the question?
- What organization do they belong to?
- What database should I query?
- What API keys should I use?

**Time:** 2-3 hours

## Setup: Install Dependencies

In [ ]:
# Install LangChain 1.0 and required packages
!pip install --pre -U langchain langchain-openai langgraph langchain-community
!pip install langgraph-checkpoint-sqlite  # For SQLite persistence

## Setup: Configure API Keys & Imports

In [ ]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Common imports
from dataclasses import dataclass
from langchain.agents import create_agent, AgentState
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool, InjectedToolArg
from typing import Annotated
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain.agents.middleware import (
    dynamic_prompt,
    before_model,
    after_model,
    ModelRequest
)
from langgraph.runtime import Runtime
import json

print("✅ Setup complete!")

---
# Part 1: Understanding Runtime & Context 🎯

## What is Runtime?

**Runtime** is the execution environment where your agent runs. It provides:
- **Context**: Static information available during execution
- **Store**: Long-term memory (covered in next module)
- **Stream Writer**: Custom streaming updates

## What is Context?

**Context** is information that **doesn't change during a conversation** but is essential for the agent to work:

```
Example without Context:
User: "Show my leave balance"
Agent: "I don't know who you are!" ❌

Example with Context:
Context: {user_id: "101", name: "Priya Sharma", dept: "Engineering"}
User: "Show my leave balance"
Agent: "Priya, you have 12 days remaining" ✅
```

## Context vs Messages

| Messages | Context |
|----------|--------|
| Changes during conversation | Static during conversation |
| User: "Hello", Agent: "Hi!" | user_id, org_id, api_keys |
| Short-term memory | Configuration |
| "What did I just say?" | "Who am I?" |

## 🔑 Critical Understanding

**The LLM does NOT automatically see context!** Context is only accessible to:
- ✅ **Tools** - via `runtime.context`
- ✅ **Dynamic Prompts** - via `request.runtime.context`
- ✅ **Middleware Hooks** - via `runtime.context`

You must **explicitly inject** context information where needed!

---
# Part 2: Basic Context Usage 📝

## Lab 1.1: Define Context Schema

In [ ]:
print("=" * 70)
print("Lab 1.1: Define Context Schema")
print("=" * 70 + "\n")

# Define what information is available in context
@dataclass
class HRContext:
    """Context for HR assistant - information about current user"""
    user_id: str
    user_name: str
    department: str
    employee_id: str
    organization_id: str

# Create sample context for testing
priya_context = HRContext(
    user_id="user_101",
    user_name="Priya Sharma",
    department="Engineering",
    employee_id="EMP-101",
    organization_id="acme_corp"
)

print("✅ Defined HRContext schema")
print(f"\nExample context for Priya:")
print(f"  User: {priya_context.user_name}")
print(f"  Department: {priya_context.department}")
print(f"  Employee ID: {priya_context.employee_id}")
print(f"  Organization: {priya_context.organization_id}")

## Lab 1.2: Agent with Context via Dynamic Prompt

**Important:** The LLM doesn't automatically see the context object. Context is only accessible to:
- ✅ Tools (via `runtime.context`)
- ✅ Dynamic prompts (via `request.runtime.context`)
- ✅ Middleware hooks (via `runtime.context`)

So we need a **dynamic prompt** to inject context info into the system message:

In [ ]:
print("=" * 70)
print("Lab 1.2: Agent with Context via Dynamic Prompt")
print("=" * 70 + "\n")

# Dynamic prompt to inject context into system message
@dynamic_prompt
def inject_user_info(request: ModelRequest) -> str:
    """Inject user information from context into system prompt."""
    user_name = request.runtime.context.user_name
    department = request.runtime.context.department
    employee_id = request.runtime.context.employee_id
    
    return f"""You are an HR assistant.
    
Current user information:
- Name: {user_name}
- Department: {department}
- Employee ID: {employee_id}

Address the user by their name and provide personalized assistance."""

# Create a simple agent with context schema
simple_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[],
    context_schema=HRContext,  # Tell agent about context structure
    middleware=[inject_user_info]  # Use dynamic prompt to inject context
)

# Invoke with Priya's context
result = simple_agent.invoke(
    {"messages": "What's my name?"},
    context=priya_context  # Pass context!
)

print("User question: What's my name?")
print(f"Agent response: {result['messages'][-1].content}")

# Try another question
result = simple_agent.invoke(
    {"messages": "What department do I work in?"},
    context=priya_context
)
print("\nUser question: What department do I work in?")
print(f"Agent response: {result['messages'][-1].content}")

print("\n✅ Agent knows user from context via dynamic prompt!")

---
# Part 3: Accessing Context in Tools 🔧

## Lab 2.1: Tool with Context Access

Tools can directly access context using `runtime.context`:

In [ ]:
print("=" * 70)
print("Lab 2.1: Tool with Context Access via Dynamic Prompt")
print("=" * 70 + "\n")

# Simple tools without runtime injection
@tool
def get_department_info() -> str:
    """Get the current user's department information.
    This tool returns the user's department based on who is logged in."""
    return "department_info"

@tool
def get_employee_id_info() -> str:
    """Get the current user's employee ID.
    This tool returns the employee ID based on who is logged in."""
    return "employee_id_info"

# Dynamic prompt that injects context AND explains tools
@dynamic_prompt
def context_aware_prompt(request: ModelRequest) -> str:
    user_name = request.runtime.context.user_name
    department = request.runtime.context.department
    employee_id = request.runtime.context.employee_id
    
    return f"""You are an HR assistant.

Current user information:
- Name: {user_name}
- Department: {department}
- Employee ID: {employee_id}

IMPORTANT: When the user asks about their department or employee ID:
- Answer directly using the information above
- You do NOT need to call any tools
- Simply tell them their department is "{department}" and their employee ID is "{employee_id}"

Be helpful and personal."""

# Create agent
agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_department_info, get_employee_id_info],
    context_schema=HRContext,
    middleware=[context_aware_prompt]
)

# Test with Priya
result = agent.invoke(
    {"messages": "What department do I work in?"},
    context=priya_context
)
print("Priya asks: What department do I work in?")
print(f"Agent: {result['messages'][-1].content}")

# Test with different user
rahul_context = HRContext(
    user_id="user_102",
    user_name="Rahul Verma",
    department="Marketing",
    employee_id="EMP-102",
    organization_id="acme_corp"
)

result = agent.invoke(
    {"messages": "What's my employee ID?"},
    context=rahul_context
)
print("\nRahul asks: What's my employee ID?")
print(f"Agent: {result['messages'][-1].content}")

print("\n✅ Same agent, different contexts - perfectly personalized!")
print("\nNote: The agent answers directly from context via dynamic prompt.")
print("Tools are available but not needed for simple context queries.")

## Lab 2.2: Tools That Use Context Information

For operations that need context (like database queries), pass user_id as a parameter:

In [ ]:
print("=" * 70)
print("Lab 2.2: Tools with User Parameters")
print("=" * 70 + "\n")

# Extended context with DB and API info
@dataclass
class ExtendedContext:
    user_id: str
    user_name: str
    department: str
    database_url: str  # Database connection info
    api_key: str  # API key for external services

# Tools that take user_id as explicit parameter
@tool
def query_leave_balance(user_id: str) -> str:
    """Query user's leave balance from database.
    
    Args:
        user_id: The user ID to query (e.g., 'user_101')
    """
    # In production, this would use context.database_url
    # For now, simulate database query
    print(f"  [Simulated] Querying database for {user_id}...")
    
    # Mock data
    leave_data = {
        "user_101": 12,
        "user_102": 8,
    }
    
    balance = leave_data.get(user_id, 0)
    return f"Leave balance for {user_id}: {balance} days"

@tool
def get_team_size(department: str) -> str:
    """Get the number of people in a department.
    
    Args:
        department: Department name (e.g., 'Engineering', 'Marketing')
    """
    dept_sizes = {
        "Engineering": 45,
        "Marketing": 20,
        "HR": 10
    }
    
    size = dept_sizes.get(department, 0)
    return f"The {department} department has {size} people."

# Dynamic prompt provides context to LLM
@dynamic_prompt  
def context_with_instructions(request: ModelRequest) -> str:
    user_id = request.runtime.context.user_id
    user_name = request.runtime.context.user_name
    department = request.runtime.context.department
    
    return f"""You are an HR assistant.

Current user:
- Name: {user_name}
- User ID: {user_id}
- Department: {department}

When user asks about THEIR leave balance, call query_leave_balance with user_id="{user_id}"
When user asks about THEIR department size, call get_team_size with department="{department}"

Always use the current user's information from above."""

agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[query_leave_balance, get_team_size],
    context_schema=ExtendedContext,
    middleware=[context_with_instructions]
)

# Create context with DB and API info
priya_extended = ExtendedContext(
    user_id="user_101",
    user_name="Priya Sharma",
    department="Engineering",
    database_url="postgresql://hr-db.acme.com:5432/employees",
    api_key="sk-acme-abc123xyz"
)

result = agent.invoke(
    {"messages": "What's my leave balance?"},
    context=priya_extended
)
print("Query 1: What's my leave balance?")
print(f"Agent: {result['messages'][-1].content}")

result = agent.invoke(
    {"messages": "How many people are in my department?"},
    context=priya_extended
)
print("\nQuery 2: How many people are in my department?")
print(f"Agent: {result['messages'][-1].content}")

print("\n✅ Tools called with correct user-specific parameters!")
print("\nKey Pattern: Dynamic prompt tells LLM which parameters to use.")

---
# Part 4: Dynamic Prompts 🎨

## Lab 3.1: Personalized System Prompts Based on Role

In [ ]:
print("=" * 70)
print("Lab 3.1: Dynamic System Prompts")
print("=" * 70 + "\n")

@dataclass
class UserContext:
    user_name: str
    department: str
    role: str  # 'employee', 'manager', 'executive'

# Dynamic prompt that changes based on context
@dynamic_prompt
def personalized_greeting(request: ModelRequest) -> str:
    """Create personalized greeting based on user role."""
    user_name = request.runtime.context.user_name
    role = request.runtime.context.role
    department = request.runtime.context.department
    
    if role == "executive":
        return f"""You are a professional HR assistant serving {user_name}, an executive. 
Be formal, concise, and provide strategic insights."""
    elif role == "manager":
        return f"""You are an HR assistant helping {user_name}, a manager in {department}. 
Be helpful and provide team management insights."""
    else:
        return f"""You are a friendly HR assistant helping {user_name} from {department}. 
Be approachable and provide clear guidance."""

agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[],
    context_schema=UserContext,
    middleware=[personalized_greeting]  # Add dynamic prompt as middleware
)

# Test with employee
employee_context = UserContext(
    user_name="Priya Sharma",
    department="Engineering",
    role="employee"
)

result = agent.invoke(
    {"messages": "How do I apply for leave?"},
    context=employee_context
)
print("Employee (Priya) asks: How do I apply for leave?")
print(f"Agent: {result['messages'][-1].content}")

# Test with executive
executive_context = UserContext(
    user_name="Mr. Kapoor",
    department="Executive Office",
    role="executive"
)

result = agent.invoke(
    {"messages": "How do I apply for leave?"},
    context=executive_context
)
print("\nExecutive (Mr. Kapoor) asks: How do I apply for leave?")
print(f"Agent: {result['messages'][-1].content}")

print("\n✅ Same question, different tones based on user role!")

---
# Part 5: Middleware - Request/Response Hooks 🔌

## Lab 4.1: Request Logging Hook

In [ ]:
print("=" * 70)
print("Lab 4.1: Request Logging Hook")
print("=" * 70 + "\n")

@dataclass
class AuditContext:
    user_id: str
    user_name: str
    organization_id: str
    session_id: str

# Before model hook - logs every request
@before_model
def audit_request(state: AgentState, runtime: Runtime[AuditContext]) -> dict | None:
    """Log request details for audit trail."""
    user_name = runtime.context.user_name
    user_id = runtime.context.user_id
    org_id = runtime.context.organization_id
    session_id = runtime.context.session_id
    
    messages = state.get('messages', [])
    last_message = messages[-1].content if messages else "N/A"
    
    print(f"\n{'='*60}")
    print(f"📝 AUDIT LOG - Request")
    print(f"{'='*60}")
    print(f"User: {user_name} ({user_id})")
    print(f"Organization: {org_id}")
    print(f"Session: {session_id}")
    print(f"Question: {last_message[:100]}..." if len(last_message) > 100 else f"Question: {last_message}")
    print(f"{'='*60}\n")
    
    return None

# After model hook - logs response
@after_model
def audit_response(state: AgentState, runtime: Runtime[AuditContext]) -> dict | None:
    """Log response details for audit trail."""
    user_name = runtime.context.user_name
    
    print(f"\n{'='*60}")
    print(f"✅ AUDIT LOG - Response")
    print(f"{'='*60}")
    print(f"Response generated for: {user_name}")
    print(f"Status: Success")
    print(f"{'='*60}\n")
    
    return None

agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[],
    context_schema=AuditContext,
    middleware=[audit_request, audit_response],  # Add both hooks
    prompt="You are a helpful HR assistant."
)

audit_context = AuditContext(
    user_id="user_101",
    user_name="Priya Sharma",
    organization_id="acme_corp",
    session_id="session_2025_001"
)

result = agent.invoke(
    {"messages": "What are the leave policies?"},
    context=audit_context
)

print("Agent response:")
print(result['messages'][-1].content)
print("\n✅ All requests and responses are logged for compliance!")

## Lab 4.2: Access Control Hook

In [ ]:
print("=" * 70)
print("Lab 4.2: Access Control Hook")
print("=" * 70 + "\n")

@dataclass
class SecurityContext:
    user_id: str
    user_name: str
    permissions: list  # List of allowed operations

@before_model
def check_permissions(state: AgentState, runtime: Runtime[SecurityContext]) -> dict | None:
    """Check if user has required permissions."""
    user_name = runtime.context.user_name
    permissions = runtime.context.permissions
    
    messages = state.get('messages', [])
    if messages:
        question = messages[-1].content.lower()
        
        # Check for restricted operations
        if 'salary' in question and 'view_salary' not in permissions:
            raise Exception(f"Access Denied: {user_name} does not have permission to view salary information.")
        
        if 'delete' in question and 'admin' not in permissions:
            raise Exception(f"Access Denied: {user_name} does not have admin permissions.")
    
    print(f"✅ Access granted for {user_name}")
    return None

agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[],
    context_schema=SecurityContext,
    middleware=[check_permissions],
    prompt="You are an HR assistant. Answer user questions."
)

# User with limited permissions
employee_sec_context = SecurityContext(
    user_id="user_101",
    user_name="Priya Sharma",
    permissions=["view_profile", "apply_leave"]
)

# Allowed question
result = agent.invoke(
    {"messages": "How do I apply for leave?"},
    context=employee_sec_context
)
print("Allowed query: How do I apply for leave?")
print(f"Agent: {result['messages'][-1].content}")

# Try restricted question
print("\nTrying restricted query: What's my salary?")
try:
    result = agent.invoke(
        {"messages": "What's my salary?"},
        context=employee_sec_context
    )
except Exception as e:
    print(f"❌ {e}")

print("\n✅ Access control enforced through middleware!")

---
# Part 6: Production Pattern - Complete Example 🎯

## Lab 5.1: Production HR Assistant with Runtime

In [ ]:
print("=" * 70)
print("Lab 5.1: Production HR Assistant")
print("=" * 70 + "\n")

# Complete production context
@dataclass
class ProductionContext:
    # User identification
    user_id: str
    user_name: str
    employee_id: str
    
    # Organization
    department: str
    role: str
    organization_id: str
    
    # Infrastructure
    database_url: str
    api_key: str
    
    # Session
    session_id: str
    permissions: list

# Production tools
@tool
def get_leave_balance(
    runtime: Annotated[Runtime[ProductionContext], InjectedToolArg]
) -> str:
    """Get current user's leave balance."""
    user_name = runtime.context.user_name
    user_id = runtime.context.user_id
    db_url = runtime.context.database_url
    
    print(f"  [Simulated] Querying {db_url}...")
    
    # Mock database query
    balances = {"user_101": 12, "user_102": 8, "user_103": 15}
    balance = balances.get(user_id, 0)
    
    return f"{user_name} has {balance} days of leave remaining."

@tool
def submit_leave_request(
    leave_type: str,
    num_days: int,
    runtime: Annotated[Runtime[ProductionContext], InjectedToolArg]
) -> str:
    """Submit a leave request."""
    user_name = runtime.context.user_name
    employee_id = runtime.context.employee_id
    
    return f"✅ Leave request submitted for {user_name} ({employee_id}): {num_days} days of {leave_type} leave."

@tool
def get_department_info(
    runtime: Annotated[Runtime[ProductionContext], InjectedToolArg]
) -> str:
    """Get information about user's department."""
    department = runtime.context.department
    org_id = runtime.context.organization_id
    
    dept_info = {
        "Engineering": "Head: Anjali Patel, Size: 45 people",
        "Marketing": "Head: Rajesh Kumar, Size: 20 people",
        "HR": "Head: Neha Singh, Size: 10 people"
    }
    
    info = dept_info.get(department, "Department information not available")
    return f"{department} Department ({org_id}): {info}"

# Dynamic prompt
@dynamic_prompt
def production_prompt(request: ModelRequest) -> str:
    user_name = request.runtime.context.user_name
    role = request.runtime.context.role
    
    base = f"You are an HR assistant helping {user_name}. "
    
    if role == "executive":
        return base + "Be formal and provide concise, strategic information."
    elif role == "manager":
        return base + "Provide helpful information for team management."
    else:
        return base + "Be friendly and provide clear, detailed guidance."

# Audit hooks
@before_model
def production_audit_log(state: AgentState, runtime: Runtime[ProductionContext]) -> dict | None:
    print(f"\n📝 Request from {runtime.context.user_name} ({runtime.context.session_id})")
    return None

@after_model
def production_response_log(state: AgentState, runtime: Runtime[ProductionContext]) -> dict | None:
    print(f"✅ Response sent to {runtime.context.user_name}\n")
    return None

# Create production agent
production_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_leave_balance, submit_leave_request, get_department_info],
    context_schema=ProductionContext,
    middleware=[production_prompt, production_audit_log, production_response_log],
    checkpointer=SqliteSaver.from_conn_string(":memory:")
)

# Create production context
priya_prod_context = ProductionContext(
    user_id="user_101",
    user_name="Priya Sharma",
    employee_id="EMP-101",
    department="Engineering",
    role="employee",
    organization_id="acme_corp",
    database_url="postgresql://hr-db.acme.com:5432/employees",
    api_key="sk-acme-prod-key-123",
    session_id="session_2025_101_001",
    permissions=["view_profile", "apply_leave", "view_department"]
)

# Test multiple queries
queries = [
    "What's my leave balance?",
    "Tell me about my department",
    "I want to apply for 3 days of sick leave"
]

for query in queries:
    result = production_agent.invoke(
        {"messages": query},
        context=priya_prod_context,
        config={"configurable": {"thread_id": "prod_thread_101"}}
    )
    print(f"Q: {query}")
    print(f"A: {result['messages'][-1].content}")
    print()

print("\n✅ Production agent with complete runtime configuration!")

---
# Summary & Key Takeaways

## What We Learned

### 1. **Runtime & Context**
```python
@dataclass
class Context:
    user_id: str
    database_url: str

agent = create_agent(
    context_schema=Context,
    ...
)

agent.invoke({...}, context=Context(...))
```

### 2. **Tools with Context-Aware Parameters**
```python
# Tools take explicit parameters
@tool
def get_user_data(user_id: str) -> str:
    return f"Data for {user_id}"

# Dynamic prompt tells LLM which values to use
@dynamic_prompt
def inject_context(request: ModelRequest) -> str:
    user_id = request.runtime.context.user_id
    return f"Current user ID is {user_id}. Use this when calling tools."
```

### 3. **Dynamic Prompts**
```python
@dynamic_prompt
def my_prompt(request: ModelRequest) -> str:
    name = request.runtime.context.user_name
    return f"You are helping {name}"
```

### 4. **Middleware Hooks**
```python
@before_model
def log_request(state: AgentState, runtime: Runtime[Context]) -> dict | None:
    # Pre-processing
    return None

@after_model
def log_response(state: AgentState, runtime: Runtime[Context]) -> dict | None:
    # Post-processing
    return None
```

## 🔑 Critical Understanding

**The LLM does NOT automatically see context!**

Context must be explicitly injected:
- ✅ **In Tools** - `runtime.context.field_name`
- ✅ **In Dynamic Prompts** - `request.runtime.context.field_name`
- ✅ **In Middleware** - `runtime.context.field_name`

## Context vs Messages vs Store

| Feature | Context | Messages | Store (Next Module) |
|---------|---------|----------|--------------------|
| **What** | Static config | Conversation | Long-term memory |
| **When** | Per invocation | During chat | Across sessions |
| **Example** | user_id, api_key | "Hello!", "Hi!" | Preferences, history |
| **Scope** | Single invocation | Single thread | All threads |
| **Access** | Tools, prompts, hooks | Always visible | Store API |

## Production Checklist

✅ **Define clear context schema** - What info does agent need?  
✅ **Use dataclasses** - Type safety and IDE support  
✅ **Inject via dynamic prompts** - For LLM to "see" context  
✅ **Access in tools** - `runtime.context.field_name`  
✅ **Dynamic prompts** - Personalize based on user role  
✅ **Audit logging** - Use before/after hooks  
✅ **Access control** - Check permissions in hooks  
✅ **Combine with checkpointer** - Short-term + Context  

## Common Patterns

### Multi-Tenant
```python
@dataclass
class TenantContext:
    user_id: str
    organization_id: str  # Separate data per org
    database_url: str  # Org-specific DB
```

### Role-Based Behavior
```python
@dynamic_prompt
def role_prompt(request: ModelRequest) -> str:
    role = request.runtime.context.role
    if role == "admin":
        return "You have admin access..."
    return "You have user access..."
```

### Audit Trail
```python
@before_model
def audit(state, runtime):
    log_to_db(
        user=runtime.context.user_id,
        query=state['messages'][-1].content,
        timestamp=now()
    )
```

## Next Module: Long-Term Memory (Store)

Now that you understand **Context** (static configuration), the next module will cover **Store** (dynamic, persistent memory):

- Storing user preferences across sessions
- Learning from past interactions
- Building knowledge over time
- Using `InjectedStore` (similar to `InjectedToolArg`)

**Key Difference:**
- **Context**: "Who is asking?" (static, per-invocation)
- **Store**: "What do I remember?" (dynamic, cross-session)

---

**Remember:** Runtime provides the environment your agent needs to work effectively!

# Exercises

## Exercise 1: Multi-Department Agent
Create an agent that:
- Has different tools for different departments
- Engineering: code deployment tools
- HR: employee management tools
- Marketing: campaign tools

## Exercise 2: Compliance Logger
Build a compliance system that:
- Logs all financial queries
- Requires approval for sensitive operations
- Generates audit reports

## Exercise 3: Multi-Language Support
Create an agent that:
- Detects user language from context
- Responds in user's preferred language
- Uses appropriate cultural references

## Exercise 4: Rate Limiting
Implement rate limiting using middleware:
- Track requests per user
- Enforce daily limits
- Provide clear error messages

## Exercise 5: Context Evolution
Design a system where:
- Context changes based on conversation
- User gets promoted → role changes
- Department transfer → permissions update